In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
%matplotlib inline
import plotly.graph_objects as go
import plotly.offline as po
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('clean_cian.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)
df

,Ссылка,Метро,Время до метро,Общая площадь,Этаж,Этажность дома,Ванная,Ремонт,Цена,Цена за метр
0,https://www.cian.ru/rent/flat/275888156/,Киевская,14.0,48.0,2.0,16.0,2,Дизайнерский,53500.0,1115.0
1,https://www.cian.ru/rent/flat/275245343/,Фили,1.0,38.0,3.0,5.0,1,Косметический,45000.0,1184.0
2,https://www.cian.ru/rent/flat/169981655/,Борисово,19.0,38.0,17.0,17.0,1,Косметический,42000.0,1105.0
3,https://www.cian.ru/rent/flat/275528458/,Верхние Лихоборы,3.0,38.0,13.0,17.0,1,Косметический,38000.0,1000.0
4,https://www.cian.ru/rent/flat/270465897/,Юго-Западная,6.0,38.0,12.0,17.0,1,Косметический,40000.0,1053.0
...,...,...,...,...,...,...,...,...,...,...
5110,https://www.cian.ru/rent/flat/226086740/,Прокшино,27.0,40.0,4.0,9.0,1,Евроремонт,38000.0,950.0
5111,https://www.cian.ru/rent/flat/275315041/,Раменки,4.0,38.0,3.0,17.0,1,Косметический,40000.0,1053.0
5112,https://www.cian.ru/rent/flat/265486228/,Нагатинская,12.0,50.0,6.0,33.0,1,Дизайнерский,65000.0,1300.0
5113,https://www.cian.ru/rent/flat/202365764/,Щелковская,25.0,44.0,3.0,14.0,1,Евроремонт,36000.0,818.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5115 entries, 0 to 5114
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ссылка          5115 non-null   object 
 1   Метро           5115 non-null   object 
 2   Время до метро  5115 non-null   float64
 3   Общая площадь   5115 non-null   float64
 4   Этаж            5115 non-null   float64
 5   Этажность дома  5115 non-null   float64
 6   Ванная          5115 non-null   int64  
 7   Ремонт          5115 non-null   object 
 8   Цена            5115 non-null   float64
 9   Цена за метр    5115 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 399.7+ KB


In [3]:
X = df.drop(['Ссылка', 'Цена', 'Цена за метр'], axis = 1)
y = df['Цена']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [4]:
cat_features = []
for col in X.columns:
    if X[col].dtype == object:
        cat_features.append(col)

cat_features

['Метро', 'Ремонт']

In [5]:
regr = CatBoostRegressor(iterations = 100000, learning_rate = 0.1, depth = 5, loss_function = 'RMSE', random_state = 42)

regr.fit(X_train, y_train, cat_features, verbose = False)
print(regr.get_best_score())

{'learn': {'RMSE': 111.43740946298499}}


In [14]:
preds = regr.predict(X_test)
np.mean(abs(preds - y_test)/y_test)

0.223484153318154

In [11]:
regr.save_model('catboost',
           format="cbm", #for mac coreml
           export_parameters=None,
           pool=None)